## Notebook for making predictions with deleted constant y samples


In [1]:
import pandas as pd
import sys
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor

KeyboardInterrupt: 

### Reading dataset stored in other notebooks

In [ ]:
sys.path.append('../')

%store -r dml

dm = dml

## Getting data

In [ ]:
print(dm.data_A.columns)

X_A_train = dm.data_A.iloc[:, 2:]
Y_A_train = dm.data_A.iloc[:, 1]

X_B_train = dm.data_B.iloc[:, 2:]
Y_B_train = dm.data_B.iloc[:, 1]

X_C_train = dm.data_C.iloc[:, 2:]
Y_C_train = dm.data_C.iloc[:, 1]

X_A_test = dm.X_test_estimated_a[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]
X_B_test = dm.X_test_estimated_b[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]
X_C_test = dm.X_test_estimated_c[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]

#X_A_train['direct_rad:W_lag_12']

Index(['date_forecast', 'pv_measurement', 'absolute_humidity_2m:gm3',
       'air_density_2m:kgm3', 'clear_sky_energy_1h:J', 'clear_sky_rad:W',
       'dew_or_rime:idx', 'dew_point_2m:K', 'diffuse_rad:W',
       'diffuse_rad_1h:J', 'direct_rad:W', 'direct_rad_1h:J',
       'effective_cloud_cover:p', 'elevation:m', 'fresh_snow_12h:cm',
       'fresh_snow_1h:cm', 'fresh_snow_24h:cm', 'fresh_snow_3h:cm',
       'fresh_snow_6h:cm', 'is_day:idx', 'is_in_shadow:idx',
       'msl_pressure:hPa', 'precip_5min:mm', 'precip_type_5min:idx',
       'pressure_100m:hPa', 'pressure_50m:hPa', 'prob_rime:p',
       'rain_water:kgm2', 'relative_humidity_1000hPa:p', 'sfc_pressure:hPa',
       'snow_depth:cm', 'snow_drift:idx', 'snow_melt_10min:mm',
       'snow_water:kgm2', 'sun_azimuth:d', 'sun_elevation:d',
       'super_cooled_liquid_water:kgm2', 't_1000hPa:K', 'total_cloud_cover:p',
       'visibility:m', 'wind_speed_10m:ms', 'wind_speed_u_10m:ms',
       'wind_speed_v_10m:ms', 'wind_speed_w_1000hPa:m

## Catboost models
will try with two models:
-   One for dataset A
-   One for B and C

In [ ]:
catModel_a = CatBoostRegressor()
catModel_b = CatBoostRegressor()
catModel_c = CatBoostRegressor()

catModel_bc = CatBoostRegressor()

#catModel_abc = CatBoostRegressor()

In [ ]:
X_BC_train = pd.concat([X_B_train, X_C_train])
Y_BC_train = pd.concat([Y_B_train, Y_C_train])

X_ABC_train = pd.concat([X_A_train, X_BC_train])
Y_ABC_train = pd.concat([Y_A_train, Y_BC_train])

In [ ]:
catModel_a.fit(X_A_train,Y_A_train)
catModel_b.fit(X_B_train,Y_B_train)
catModel_c.fit(X_C_train,Y_C_train)

#catModel_bc.fit(X_BC_train,Y_BC_train)

#catModel_abc.fit(X_ABC_train,Y_ABC_train)

Learning rate set to 0.059717
0:	learn: 1408.7522677	total: 4.55ms	remaining: 4.55s
1:	learn: 1345.9811388	total: 9.77ms	remaining: 4.88s
2:	learn: 1286.8438117	total: 13.6ms	remaining: 4.52s
3:	learn: 1232.9551590	total: 17.3ms	remaining: 4.3s
4:	learn: 1181.2767859	total: 21ms	remaining: 4.19s
5:	learn: 1132.5233095	total: 25.2ms	remaining: 4.18s
6:	learn: 1087.2843095	total: 28.9ms	remaining: 4.09s
7:	learn: 1046.0153469	total: 32.3ms	remaining: 4s
8:	learn: 1008.1312367	total: 36ms	remaining: 3.96s
9:	learn: 972.1252161	total: 39.5ms	remaining: 3.91s
10:	learn: 938.7715551	total: 43.2ms	remaining: 3.88s
11:	learn: 907.7824862	total: 46.7ms	remaining: 3.85s
12:	learn: 878.6876102	total: 50.4ms	remaining: 3.83s
13:	learn: 852.5943142	total: 54.7ms	remaining: 3.85s
14:	learn: 827.8656078	total: 58.7ms	remaining: 3.85s
15:	learn: 805.5168376	total: 62.3ms	remaining: 3.83s
16:	learn: 784.8085207	total: 66.4ms	remaining: 3.84s
17:	learn: 766.6397917	total: 70ms	remaining: 3.82s
18:	learn

In [ ]:
# Feature importance
catModel_a.get_feature_importance(prettified=True)

,Feature Id,Importances
0,direct_rad:W,20.710842
1,clear_sky_rad:W,9.226983
2,direct_rad_1h:J,7.742377
3,diffuse_rad:W,6.824410
4,sun_azimuth:d,4.859122
5,sun_elevation:d,4.369468
6,wind_speed_u_10m:ms,4.065331
7,diffuse_rad_1h:J,3.780212
8,day_year,3.593923
9,direct_rad:W_lag_-23,3.364169


## Preds

In [ ]:
pred_A = catModel_a.predict(X_A_test)
pred_B = catModel_b.predict(X_B_test)
pred_C = catModel_c.predict(X_C_test)

# pred_B = catModel_bc.predict(X_B_test)
# pred_C = catModel_bc.predict(X_C_test)

# pred_A = catModel_abc.predict(X_A_test)
# pred_B = catModel_abc.predict(X_B_test)
# pred_C = catModel_abc.predict(X_C_test)

In [ ]:
df_A = pd.DataFrame()

df_A["predict"] = pred_A
df_A["location"] = "A"

df_B = pd.DataFrame()

df_B["predict"] = pred_B
df_B["location"] = "B"

df_C = pd.DataFrame()

df_C["predict"] = pred_C
df_C["location"] = "C"

df_mid = pd.concat([df_A, df_B], ignore_index=True)

df = pd.concat([df_mid, df_C], join="inner", ignore_index=True)

df = df.drop("location", axis=1)

df[df<0] = 0

# NAME THE FILE 
#df.to_csv("./ext_preds/catboost_23_sep.csv")

df.shape

(2160, 1)